# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1QHOQt1J7d5Z4sBFKL9GN699OcPh2aVaP?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver
No Chrome: ChromeDriver

In [8]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# pyautogui automatiza mouse, tela e teclado, mas não podemos usar o computador enquanto a automação está em uso para não atrapalhar a configuração. Ponto positivo é que funciona de qualquer ferramenta/sistema/programa;
# selenium automatiza por meio da internet, funciona em segundo plano e mesmo que as posições das coisas na tela mudarem

In [42]:
navegador = webdriver.Chrome() 

In [44]:
#cotação do dólar
navegador.get("https://www.google.com/")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#como pegar o local onde aparece o valor do dólar:
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

# pegar cotação do euro
navegador.get("https://www.google.com/")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#como pegar o local onde aparece o valor do euro:
cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

# pegar cotacao ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
navegador.find_element_by_xpath('//*[@id="commodity-hoje"]/tbody/tr[2]/td[2]/a/img').click()
#como pegar o local onde aparece o valor do ouro:
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".") #mudar a , por . no valor da cotação do ouro
print(cotacao_ouro)


5.438336
6.536036929000001
309.14


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [45]:
import pandas as pd

tabela_produtos = pd.read_excel('Produtos_aula4.xlsx')
display(tabela_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [39]:
# é preciso atualizar as cotações em dólar, euro e ouro:
tabela_produtos.loc[tabela_produtos["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela_produtos.loc[tabela_produtos["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela_produtos.loc[tabela_produtos["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualizar o Preço Base Reais para (Cotação * Preço Base Original)
tabela_produtos["Preço Base Reais"] = tabela_produtos['Cotação'] * tabela_produtos['Preço Base Original']

# atualizar o Preço Final para (Preço Base Reais * Ajuste)
tabela_produtos["Preço Final"] = tabela_produtos['Preço Base Reais'] * tabela_produtos['Ajuste']
tabela_produtos["Preço Final"] = tabela_produtos["Preço Final"].map("{:.2f}".format) # a partir do .map usamos código para formatar o Preço Final com duas casas decimais

display(tabela_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5.438336,5438.281617,1.40,7613.59
1,Carro Renault,4500.00,Euro,6.536603,29414.711322,2.00,58829.42
2,Notebook Dell,899.99,Dólar,5.438336,4894.448017,1.70,8320.56
3,IPhone,799.00,Dólar,5.438336,4345.230464,1.70,7386.89
4,Carro Fiat,3000.00,Euro,6.536603,19609.807548,1.90,37258.63
5,Celular Xiaomi,480.48,Dólar,5.438336,2613.011681,2.00,5226.02
6,Joia 20g,20.00,Ouro,309.140000,6182.800000,1.15,7110.22


### Agora vamos exportar a nova base de preços atualizada

In [47]:
tabela_produtos.to_excel("Produtos Atualizado_aula4.xlsx", index = False) #index = False significa que na planilha exportada não aparecerá a 1ª coluna, como acima,que tem o índice de cada linha da tabela 